# Pods

Podman has the hability to run containers in Pods, like in Kubernetes.

Pod is a concept in which a group of one or more containers working together for a common purpose and sharing the same namespaces and cgroups (resource constraints).
This way it is also possible to group containers's network space and volume mounts, making it easier for the containers to interact and work as if they were a single unit.


## Running Pods

To run containers in pods requires to plan a little bit, as the pod needs to be created up-front, specifying the volumes and ports that will be required.
Afterwards the containers will be attached to this pod.

We will run the following example to demonstrate how it works, start a simple http server in one pod, while in another we will refresh the index.html using a volume: 2 concerns (2 containers and 1 pod):

1. Create a pod
1. Create the web server container and attach it to the pod
1. Create the sidecar container that will update the index.html



### Create the Pod

After creating the path for the volume mount we will:

1. Specify the port binding (8080:80)
1. Specify a volume mount to `/usr/local/apache2/htdocs` with the `z` parameter. `z` specifies that the volume mount should happen in each container (shared).

In [2]:
podman volume create webserver-data

: 1

In [11]:
podman pod create -p 8080:80 --name dynamic-webserver --volume  webserver-data:/usr/local/apache2/htdocs:z

d73f9bc69c6c33ff0b8bcfef53b85ad8300393e2a179621520553bfc004517c2


: 1

In [12]:
podman pod ps

POD ID        NAME               STATUS      CREATED       INFRA ID      # OF CONTAINERS
d73f9bc69c6c  dynamic-webserver  Created     1 second ago  6ff3524a368e  1


: 1

### Inspecting the pod (right after creation)

Although we just created a pod without explicitly attaching containers to it, there is already a running container in this pod: the `pause` container from the Kubernetes project.
It is in charge of maintaining the namespaces and networks from the whole pod.

Let's check this out:

In [13]:
podman ps --pod

CONTAINER ID  IMAGE                                    COMMAND     CREATED         STATUS             PORTS                 NAMES               POD ID        PODNAME
4b9f9d36290a  localhost/podman-pause:4.1.1-1655914710              18 minutes ago  Up 18 minutes ago  0.0.0.0:8080->80/tcp  eb28e60a4641-infra  eb28e60a4641  dynamic-webserver


: 1

In [14]:
podman inspect $(podman ps | awk '/pause/{ print $NF }' | grep infra) --format '{{.Mounts}}'

[{volume webserver-data /var/lib/containers/storage/volumes/webserver-data/_data /usr/local/apache2/htdocs local z [nosuid nodev rbind] true rprivate}]


: 1

In [15]:
podman inspect $(podman ps | awk '/pause/{ print $NF }' | grep infra) --format '{{.HostConfig.PortBindings}}'

map[80/tcp:[{ 8080}]]


: 1

### Attach the containers

1. First we create the httpd container, which simply serves the index.html file from the htdocs folder
1. Then we start a sidecar container that will update the index adding timestamps. This is a silly test whose purpose is to demonstrate the volume share.

Note that we are not supplying ports nor volumes to the single containers, they inherit those from the pod.

In [16]:
podman run -d --name apache-httpd --pod dynamic-webserver httpd:2.4

37303e17edb06c20421084c116fd097e35163bc9c9c7eed381985aa6a395c540


: 1

In [17]:
curl -s localhost:8080

<html><body><h1>It works!</h1></body></html>
Index Updated @Mon Jul 25 08:08:12 UTC 2022
Index Updated @Mon Jul 25 08:08:22 UTC 2022
Index Updated @Mon Jul 25 08:08:32 UTC 2022
Index Updated @Mon Jul 25 08:08:42 UTC 2022
Index Updated @Mon Jul 25 08:08:52 UTC 2022
Index Updated @Mon Jul 25 08:09:02 UTC 2022
Index Updated @Mon Jul 25 08:09:12 UTC 2022
Index Updated @Mon Jul 25 08:09:22 UTC 2022
Index Updated @Mon Jul 25 08:09:32 UTC 2022
Index Updated @Mon Jul 25 08:09:42 UTC 2022
Index Updated @Mon Jul 25 08:09:52 UTC 2022
Index Updated @Mon Jul 25 08:10:02 UTC 2022
Index Updated @Mon Jul 25 08:10:12 UTC 2022
Index Updated @Mon Jul 25 08:10:22 UTC 2022
Index Updated @Mon Jul 25 08:10:32 UTC 2022
Index Updated @Mon Jul 25 08:10:42 UTC 2022
Index Updated @Mon Jul 25 08:10:52 UTC 2022
Index Updated @Mon Jul 25 08:11:02 UTC 2022
Index Updated @Mon Jul 25 08:11:12 UTC 2022
Index Updated @Mon Jul 25 08:11:22 UTC 2022
Index Updated @Mon Jul 25 08:11:32 UTC 2022
Index Updated @Mon Jul 25 08:11

: 1

In [ ]:
podman run -d --name apache-httpd-index-updater --pod dynamic-webserver alpine sh -c 'while true; do echo "Index Updated @$(date)" >> /usr/local/apache2/htdocs/index.html; sleep 10; done'

## Cleanup Pods

Removing pods deletes also their running containers, which is a fast way to cleanup resources.

The command is pretty much the same as with containers, but prefixed with the word `pod`:

In [10]:
podman pod rm -f dynamic-webserver

923bf3dcbea904e915711af640db2b4f6eaab3235639552fde0bc7b8036b8a74


: 1